In [1]:
import numpy as np
import numba as nb
import torch

from Recherches.LinearRepresentations.shuffle import get_shuffle_operator
from Recherches.LinearRepresentations.models import linear_form_ncir, make_linear_form_timedep

ModuleNotFoundError: No module named 'torch'

In [629]:
%%time
dim = 2
trunc = 6
max_trunc = trunc
shuffle_operator = get_shuffle_operator(
    left_dim=dim, right_dim=dim,
    left_trunc=trunc, right_trunc=trunc,
    max_trunc=max_trunc,
)
shuffle_operator

• Start: Fri Dec 13 2024, 11h23


array([[  0,   0,   0,   1],
       [  1,   0,   1,   1],
       [  1,   1,   0,   1],
       ...,
       [126,  30,   6,  15],
       [126,  62,   2,   6],
       [126, 126,   0,   1]])

• End  : Fri Dec 13 2024, 11h23 (3.8 ms)


In [630]:
print((dim ** (max_trunc + 1) - 1) // (dim - 1), shuffle_operator[-1, 0] + 1)
print((dim ** (trunc + 1) - 1) // (dim - 1), shuffle_operator[-1, 1] + 1, shuffle_operator[-1, 2] + 1)

127 127
127 127 1


In [631]:
%%time
V_0 = 0.2
kappa = 4.
theta = 0.25
eta = 1.2

n = 1   # power of ell
ell = linear_form_ncir(trunc=trunc, V_0=V_0, kappa=kappa, theta=theta, eta=eta, n=n)
ell_squared = linear_form_ncir(trunc=max_trunc, V_0=V_0, kappa=kappa, theta=theta, eta=eta, n=n*2)
ell_squared

• Start: Fri Dec 13 2024, 11h23


LinearFunctional(np.array([ 2.00000000e-01, -1.60000000e-01,  5.36656315e-01,  6.40000000e-01,
       -2.14662526e-01, -2.14662526e+00,  7.20000000e-01, -2.56000000e+00,
        7.72785093e-01,  8.58650103e-01,  0.00000000e+00,  8.58650103e+00,
       -2.59200000e+00, -2.88000000e+00,  0.00000000e+00,  1.02400000e+01,
       -2.50725830e+00, -3.09114037e+00,  0.00000000e+00, -3.43460041e+00,
       -9.21600000e-01,  0.00000000e+00,  0.00000000e+00, -3.43460041e+01,
        8.40960000e+00,  1.03680000e+01,  0.00000000e+00,  1.15200000e+01,
        3.09114037e+00,  0.00000000e+00,  0.00000000e+00, -4.09600000e+01,
        6.38835677e+00,  1.00290332e+01,  0.00000000e+00,  1.23645615e+01,
        2.21184000e+00,  0.00000000e+00,  0.00000000e+00,  1.37384017e+01,
        8.84736000e+00,  3.68640000e+00,  0.00000000e+00,  0.00000000e+00,
        3.70936845e+00,  0.00000000e+00,  0.00000000e+00,  1.37384017e+02,
       -2.14272000e+01, -3.36384000e+01,  0.00000000e+00, -4.14720000e+01,
     

• End  : Fri Dec 13 2024, 11h23 (12.4 ms)


In [632]:
@nb.jit
def shuffle_product_numba(linear_left, linear_right, shuop):
    index_result, index_left, index_right, count = shuop.T
    source = count * linear_left[index_left] * linear_right[index_right]
    linear_result = np.zeros(shape=index_result[-1]+1)
    for i in range(len(index_result)):
        linear_result[index_result[i]] += source[i]
    return linear_result

def shuffle_product_numpy(linear_left, linear_right, shuop):
    index_result, index_left, index_right, count = shuop.T
    source = count * linear_left[index_left] * linear_right[index_right]
    linear_result = np.zeros(shape=index_result[-1]+1)
    np.add.at(linear_result, index_result, source)
    return linear_result

def shuffle_product_torch(linear_left, linear_right, shuop):
    index_result, index_left, index_right, count = shuop.T
    source = count * linear_left[index_left] * linear_right[index_right]
    linear_result = torch.zeros(size=(index_result[-1]+1,), dtype=source.dtype, device=source.device)
    linear_result.scatter_add_(dim=0, index=index_result, src=source)
    return linear_result

In [633]:
ell_squared_numba = shuffle_product_numba(ell.array, ell.array, shuffle_operator)
ell_squared_numba

array([ 2.00000000e-01, -1.60000000e-01,  5.36656315e-01,  6.40000000e-01,
       -2.14662526e-01, -2.14662526e+00,  7.20000000e-01, -2.56000000e+00,
        7.72785093e-01,  8.58650103e-01,  0.00000000e+00,  8.58650103e+00,
       -2.59200000e+00, -2.88000000e+00,  0.00000000e+00,  1.02400000e+01,
       -2.50725830e+00, -3.09114037e+00,  0.00000000e+00, -3.43460041e+00,
       -9.21600000e-01, -6.66133815e-16,  0.00000000e+00, -3.43460041e+01,
        8.40960000e+00,  1.03680000e+01,  0.00000000e+00,  1.15200000e+01,
        3.09114037e+00, -1.77635684e-15,  0.00000000e+00, -4.09600000e+01,
        6.38835677e+00,  1.00290332e+01,  0.00000000e+00,  1.23645615e+01,
        2.21184000e+00,  1.77635684e-15,  0.00000000e+00,  1.37384017e+01,
        8.84736000e+00,  3.68640000e+00,  0.00000000e+00,  3.90798505e-14,
        3.70936845e+00, -1.50990331e-14,  0.00000000e+00,  1.37384017e+02,
       -2.14272000e+01, -3.36384000e+01,  0.00000000e+00, -4.14720000e+01,
       -7.41873689e+00, -

In [634]:
ell_squared_numpy = shuffle_product_numpy(ell, ell, shuffle_operator)
ell_squared_numpy

array([ 2.00000000e-01, -1.60000000e-01,  5.36656315e-01,  6.40000000e-01,
       -2.14662526e-01, -2.14662526e+00,  7.20000000e-01, -2.56000000e+00,
        7.72785093e-01,  8.58650103e-01,  0.00000000e+00,  8.58650103e+00,
       -2.59200000e+00, -2.88000000e+00,  0.00000000e+00,  1.02400000e+01,
       -2.50725830e+00, -3.09114037e+00,  0.00000000e+00, -3.43460041e+00,
       -9.21600000e-01, -6.66133815e-16,  0.00000000e+00, -3.43460041e+01,
        8.40960000e+00,  1.03680000e+01,  0.00000000e+00,  1.15200000e+01,
        3.09114037e+00, -1.77635684e-15,  0.00000000e+00, -4.09600000e+01,
        6.38835677e+00,  1.00290332e+01,  0.00000000e+00,  1.23645615e+01,
        2.21184000e+00,  1.77635684e-15,  0.00000000e+00,  1.37384017e+01,
        8.84736000e+00,  3.68640000e+00,  0.00000000e+00,  3.90798505e-14,
        3.70936845e+00, -1.50990331e-14,  0.00000000e+00,  1.37384017e+02,
       -2.14272000e+01, -3.36384000e+01,  0.00000000e+00, -4.14720000e+01,
       -7.41873689e+00, -

In [635]:
torch.set_default_device("cpu") # "cuda" if you have a nvidia GPU
ell_torch = torch.from_numpy(ell.array).double()
shuffle_operator_torch = torch.from_numpy(shuffle_operator).long()
ell_squared_torch = shuffle_product_torch(ell_torch, ell_torch, shuffle_operator_torch).numpy()
ell_squared_torch

array([ 2.00000000e-01, -1.60000000e-01,  5.36656315e-01,  6.40000000e-01,
       -2.14662526e-01, -2.14662526e+00,  7.20000000e-01, -2.56000000e+00,
        7.72785093e-01,  8.58650103e-01,  0.00000000e+00,  8.58650103e+00,
       -2.59200000e+00, -2.88000000e+00,  0.00000000e+00,  1.02400000e+01,
       -2.50725830e+00, -3.09114037e+00,  0.00000000e+00, -3.43460041e+00,
       -9.21600000e-01, -6.66133815e-16,  0.00000000e+00, -3.43460041e+01,
        8.40960000e+00,  1.03680000e+01,  0.00000000e+00,  1.15200000e+01,
        3.09114037e+00, -1.77635684e-15,  0.00000000e+00, -4.09600000e+01,
        6.38835677e+00,  1.00290332e+01,  0.00000000e+00,  1.23645615e+01,
        2.21184000e+00,  1.77635684e-15,  0.00000000e+00,  1.37384017e+01,
        8.84736000e+00,  3.68640000e+00,  0.00000000e+00,  3.90798505e-14,
        3.70936845e+00, -1.50990331e-14,  0.00000000e+00,  1.37384017e+02,
       -2.14272000e+01, -3.36384000e+01,  0.00000000e+00, -4.14720000e+01,
       -7.41873689e+00, -

In [636]:
print(np.allclose(ell_squared, ell_squared_numba))
print(np.allclose(ell_squared, ell_squared_numpy))
print(np.allclose(ell_squared, ell_squared_torch))

True
True
True


In [348]:
%timeit shuffle_product_numba(ell.array, ell.array, shuffle_operator)

11.3 μs ± 77.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [349]:
%timeit shuffle_product_numpy(ell, ell, shuffle_operator)

16.3 μs ± 144 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [350]:
%timeit shuffle_product_torch(ell_torch, ell_torch, shuffle_operator_torch)

34.4 μs ± 164 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [637]:
T = 10
N = 1000
t = np.linspace(start=0, stop=T, num=N)

sigma = make_linear_form_timedep(ell, t=t, region=theta, beta=kappa, shuop=shuffle_operator)

In [638]:
@nb.jit(parallel=True)
def shuffle_product_batch_numba(linear_left, linear_right, shuop):
    linear_result = np.empty(shape=(N, shuop[-1, 0]+1))
    for j in nb.prange(linear_left.shape[0]):
        linear_result[j] = shuffle_product_numba(linear_left[j], linear_right[j], shuop)
    return linear_result

def shuffle_product_batch_numpy(linear_left, linear_right, shuop):
    index_result, index_left, index_right, count = shuop.T
    source = count * linear_left[:, index_left] * linear_right[:, index_right]
    linear_result = np.zeros(shape=(index_result[-1]+1, source.shape[0]))
    np.add.at(linear_result, index_result, source.T)
    return linear_result.T

def shuffle_product_batch_torch(linear_left, linear_right, shuop):
    index_result, index_left, index_right, count = shuop.T
    source = count * linear_left[:, index_left] * linear_right[:, index_right]
    linear_result = torch.zeros(size=(source.shape[0], index_result[-1]+1), dtype=source.dtype, device=source.device)
    index_result = torch.tile(index_result, dims=(source.shape[0], 1))
    linear_result.scatter_add_(dim=1, index=index_result, src=source)
    return linear_result

In [639]:
sigma_squared_numba = shuffle_product_batch_numba(sigma.array, sigma.array, shuffle_operator)
sigma_squared_numba

array([[ 2.00000000e-01,  5.45572809e-01,  5.36656315e-01, ...,
         6.67686320e+01,  5.68434189e-14,  0.00000000e+00],
       [ 1.93136652e-01,  5.15087370e-01,  5.06669110e-01, ...,
         6.16302801e+01, -7.03731927e+00,  0.00000000e+00],
       [ 1.86655551e-01,  4.86496631e-01,  4.78545640e-01, ...,
         5.68873633e+01, -1.32568556e+01,  0.00000000e+00],
       ...,
       [ 6.25000000e-02,  1.40370804e-18,  1.38076673e-18, ...,
         1.41439463e-33, -8.58947370e-16,  0.00000000e+00],
       [ 6.25000000e-02,  1.34861387e-18,  1.32657298e-18, ...,
         1.30554625e-33, -8.25234519e-16,  0.00000000e+00],
       [ 6.25000000e-02,  1.29568208e-18,  1.27450628e-18, ...,
         1.20507458e-33, -7.92844865e-16,  0.00000000e+00]])

In [640]:
sigma_squared_numpy = shuffle_product_batch_numpy(sigma, sigma, shuffle_operator)
sigma_squared_numpy

array([[ 2.00000000e-01,  5.45572809e-01,  5.36656315e-01, ...,
         6.67686320e+01,  5.68434189e-14,  0.00000000e+00],
       [ 1.93136652e-01,  5.15087370e-01,  5.06669110e-01, ...,
         6.16302801e+01, -7.03731927e+00,  0.00000000e+00],
       [ 1.86655551e-01,  4.86496631e-01,  4.78545640e-01, ...,
         5.68873633e+01, -1.32568556e+01,  0.00000000e+00],
       ...,
       [ 6.25000000e-02,  1.40370804e-18,  1.38076673e-18, ...,
         1.41439463e-33, -8.58947370e-16,  0.00000000e+00],
       [ 6.25000000e-02,  1.34861387e-18,  1.32657298e-18, ...,
         1.30554625e-33, -8.25234519e-16,  0.00000000e+00],
       [ 6.25000000e-02,  1.29568208e-18,  1.27450628e-18, ...,
         1.20507458e-33, -7.92844865e-16,  0.00000000e+00]])

In [646]:
torch.set_default_device("cpu")     # "cuda" if you have a nvidia GPU
sigma_torch = torch.from_numpy(sigma.array).double()
shuffle_operator_torch = torch.from_numpy(shuffle_operator).long()
sigma_squared_torch = shuffle_product_batch_torch(sigma_torch, sigma_torch, shuffle_operator_torch).numpy()
sigma_squared_torch

array([[ 2.00000000e-01,  5.45572809e-01,  5.36656315e-01, ...,
         6.67686320e+01,  5.68434189e-14,  0.00000000e+00],
       [ 1.93136652e-01,  5.15087370e-01,  5.06669110e-01, ...,
         6.16302801e+01, -7.03731927e+00,  0.00000000e+00],
       [ 1.86655551e-01,  4.86496631e-01,  4.78545640e-01, ...,
         5.68873633e+01, -1.32568556e+01,  0.00000000e+00],
       ...,
       [ 6.25000000e-02,  1.40370804e-18,  1.38076673e-18, ...,
         1.41439463e-33, -8.58947370e-16,  0.00000000e+00],
       [ 6.25000000e-02,  1.34861387e-18,  1.32657298e-18, ...,
         1.30554625e-33, -8.25234519e-16,  0.00000000e+00],
       [ 6.25000000e-02,  1.29568208e-18,  1.27450628e-18, ...,
         1.20507458e-33, -7.92844865e-16,  0.00000000e+00]])

In [647]:
print(np.allclose(sigma_squared_numba, sigma_squared_numpy))
print(np.allclose(sigma_squared_numba, sigma_squared_torch))

True
True


In [438]:
%timeit shuffle_product_batch_numba(sigma.array, sigma.array, shuffle_operator)

1.45 ms ± 9.79 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [439]:
%timeit shuffle_product_batch_numpy(sigma, sigma, shuffle_operator)

16 ms ± 138 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [648]:
%timeit shuffle_product_batch_torch(sigma_torch, sigma_torch, shuffle_operator_torch).numpy()

2.63 ms ± 89.3 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [564]:
from itertools import combinations
from functools import cache

def concatenate(words, letter: str):
    return [word + letter for word in words]

@cache
def riffle_shuffle(left_word: str, right_word: str):
    if left_word in ["", "ø", "0"]:
        return [right_word]
    if right_word in ["", "ø", "0"]:
        return [left_word]

    u, i = left_word[:-1], left_word[-1]
    v, j = right_word[:-1], right_word[-1]

    left_half_shuffle = concatenate(words=riffle_shuffle(left_word=u, right_word=right_word), letter=i)
    right_half_shuffle = concatenate(words=riffle_shuffle(left_word=left_word, right_word=v), letter=j)

    return [*left_half_shuffle, *right_half_shuffle]

In [661]:
u = "12"
v = "34"
w = u+v
u_v_str = sorted(riffle_shuffle(u, v))
u_v_str

['1234', '1324', '1342', '3124', '3142', '3412']

In [662]:
p, q = len(u), len(v)
indices = np.concatenate([
    np.stack([*combinations(iterable=np.arange(p+q), r=p)], axis=0),
    np.stack([*combinations(iterable=np.arange(p+q), r=q)], axis=0)[::-1],
], axis=1)
indices

array([[0, 1, 2, 3],
       [0, 2, 1, 3],
       [0, 3, 1, 2],
       [1, 2, 0, 3],
       [1, 3, 0, 2],
       [2, 3, 0, 1]])

In [663]:
uv = np.array([*w]).astype(int)
uv

array([1, 2, 3, 4])

In [664]:
powers = p+q-1 - indices
u_v_int = np.sum(uv * (10 ** powers), axis=1)
u_v_int

array([1234, 1324, 1342, 3124, 3142, 3412])